In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
from SimpleCNN18K import SimpleCNN18K   

Assign the Device

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else 'cpu')
print(device)

cuda


Download the data

In [3]:
torch.manual_seed(1)
batch_size = 16

kwargs = {'num_workers':1,'pin_memory':True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=True,download=True,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,),(0.3081,))]
                   )),batch_size=batch_size,shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=False,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,),(0.3081,))
                                                 ])),batch_size=batch_size,shuffle=True,**kwargs)

Train and Test Functions

In [4]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  train_loss = 0
  correct = 0
  pbar = tqdm(train_loader)
  for batch_idx, (data,target) in enumerate(pbar):
    data,target = data.to(device),target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output,target)
    loss.backward()
    optimizer.step()
    pbar.set_description(desc=f'loss={loss.item()} batch_id={batch_idx}')
    with torch.no_grad():
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
  train_loss /= len(train_loader.dataset)
  print('\n Epoch No: {}'.format(epoch))
  print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
    train_loss, correct, len(train_loader.dataset),
    100. * float(correct) / len(train_loader.dataset)))

def test(model, device, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data,target in test_loader:
      data,target = data.to(device),target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output,target,reduction='sum').item()
      pred = output.argmax(dim=1,keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  print('Test set: Average loss: {:.4f}, Accuracy:{}/{} ({:.2f}%)\n'.format(
      test_loss,correct,len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))

Define the CNN model with different archicture and check for expected results

Version 1

In [5]:
class Net_v1(nn.Module):
  def __init__(self):
    super(Net_v1,self).__init__()
    self.conv1 = nn.Conv2d(1,8,3,stride=1,padding=1) # input_size = 28, output_size = 28, RF = 3, No_param =
    self.conv2 = nn.Conv2d(8,8,3,stride=1,padding=1)# input_size = 28, output_size = 28, RF = 5
    self.pool1 = nn.MaxPool2d(2,2)                    # input_size = 28, output_size = 14, RF = 6
    self.conv3 = nn.Conv2d(8,16,3,stride=1,padding=1)# input_size = 14, output_size = 14, RF = 10
    self.conv4 = nn.Conv2d(16,16,3,stride=1,padding=1)# input_size = 14, output_size = 14, RF = 14
    self.pool2 = nn.MaxPool2d(2,2)                      # input_size = 14, output_size = 7, RF = 16
    self.conv5 = nn.Conv2d(16,32,3,stride=1,padding=0)# input_size = 7, output_size = 5, RF = 24
    self.conv6 = nn.Conv2d(32,32,3,stride=1,padding=0)# input_size = 5, output_size = 3, RF = 32
    self.conv7 = nn.Conv2d(32,10,3)                    # input_size = 3, output_size = 1, RF = 40

  def forward(self,x):
    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))
    x = self.conv7(x)
    x = x.view(-1,10)
    return F.log_softmax(x)

In [6]:
model = Net_v1().to(device)
summary(model,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
            Conv2d-2            [-1, 8, 28, 28]             584
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
            Conv2d-5           [-1, 16, 14, 14]           2,320
         MaxPool2d-6             [-1, 16, 7, 7]               0
            Conv2d-7             [-1, 32, 5, 5]           4,640
            Conv2d-8             [-1, 32, 3, 3]           9,248
            Conv2d-9             [-1, 10, 1, 1]           2,890
Total params: 20,930
Trainable params: 20,930
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.17
Params size (MB): 0.08
Estimated Total Size (MB): 0.25
---------------------------------------------

C:\Users\ic016726\AppData\Local\Temp\ipykernel_28120\948642335.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:
model = Net_v1().to(device)
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.9)

for epoch in range(1,10):
  train(model,device, train_loader, optimizer, epoch)
  test(model,device, train_loader)

  0%|          | 0/3750 [00:00<?, ?it/s]C:\Users\ic016726\AppData\Local\Temp\ipykernel_28120\948642335.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.002319438150152564 batch_id=3749: 100%|██████████| 3750/3750 [00:28<00:00, 133.03it/s]  



 Epoch No: 1
Train set: Average loss: 0.0000, Accuracy: 56243/60000 (93.74%)
Test set: Average loss: 0.0927, Accuracy:58347/60000 (97.25%)



loss=0.07344766706228256 batch_id=3749: 100%|██████████| 3750/3750 [00:27<00:00, 134.47it/s]   



 Epoch No: 2
Train set: Average loss: 0.0000, Accuracy: 59265/60000 (98.78%)
Test set: Average loss: 0.0493, Accuracy:59096/60000 (98.49%)



loss=4.8721933126216754e-05 batch_id=3749: 100%|██████████| 3750/3750 [00:27<00:00, 136.28it/s]


...


 Epoch No: 7
Train set: Average loss: 0.0000, Accuracy: 59732/60000 (99.55%)
Test set: Average loss: 0.0216, Accuracy:59585/60000 (99.31%)



loss=0.04937540739774704 batch_id=3749: 100%|██████████| 3750/3750 [00:26<00:00, 139.68it/s]   



 Epoch No: 8
Train set: Average loss: 0.0000, Accuracy: 59764/60000 (99.61%)
Test set: Average loss: 0.0205, Accuracy:59603/60000 (99.34%)



loss=0.009236565791070461 batch_id=3749: 100%|██████████| 3750/3750 [00:23<00:00, 156.56it/s]  



 Epoch No: 9
Train set: Average loss: 0.0000, Accuracy: 59780/60000 (99.63%)
Test set: Average loss: 0.0335, Accuracy:59340/60000 (98.90%)



Version 2

In [8]:
class Net_v2(nn.Module):
  def __init__(self):
    super(Net_v2,self).__init__()
    self.conv1 = nn.Conv2d(1,8,3,stride=1,padding=1) # input_size = 28, output_size = 28, RF = 3,
    self.conv1_bn = nn.BatchNorm2d(8)
    self.conv2 = nn.Conv2d(8,16,3,stride=1,padding=0)# input_size = 28, output_size = 26, RF = 5
    self.conv2_bn = nn.BatchNorm2d(16)
    self.pool1 = nn.MaxPool2d(2,2)                    # input_size = 26, output_size = 13, RF = 6
    self.conv3 = nn.Conv2d(16,16,3,stride=1,padding=1)# input_size = 13, output_size = 13, RF = 10
    self.conv3_bn = nn.BatchNorm2d(16)
    self.conv4 = nn.Conv2d(16,32,3,stride=1,padding=0)# input_size = 13, output_size = 11, RF = 14
    self.conv4_bn = nn.BatchNorm2d(32)
    self.pool2 = nn.MaxPool2d(2,2)                      # input_size = 11, output_size = 5, RF = 16
    self.conv5 = nn.Conv2d(32,32,3,stride=1,padding=0)# input_size = 5, output_size = 3, RF = 24
    self.conv5_bn = nn.BatchNorm2d(32)
    self.pool3 = nn.AdaptiveAvgPool2d((1, 1))  # reduces the spatial dimensions to 1x1
    self.conv6 = nn.Conv2d(32,10,1)                   # input_size = 3, output_size = 1, RF = 40

  def forward(self,x):
    x = self.pool1(F.relu(self.conv2_bn(self.conv2(F.relu(self.conv1_bn(self.conv1(x)))))))
    x = self.pool2(F.relu(self.conv4_bn(self.conv4(F.relu(self.conv3_bn(self.conv3(x)))))))
    x = self.pool3(F.relu(self.conv5_bn(self.conv5(x))))
    x = self.conv6(x)
    x = x.view(-1,10)
    return F.log_softmax(x)

In [9]:
model_2 = Net_v2().to(device)
summary(model_2,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 26, 26]           1,168
       BatchNorm2d-4           [-1, 16, 26, 26]              32
         MaxPool2d-5           [-1, 16, 13, 13]               0
            Conv2d-6           [-1, 16, 13, 13]           2,320
       BatchNorm2d-7           [-1, 16, 13, 13]              32
            Conv2d-8           [-1, 32, 11, 11]           4,640
       BatchNorm2d-9           [-1, 32, 11, 11]              64
        MaxPool2d-10             [-1, 32, 5, 5]               0
           Conv2d-11             [-1, 32, 3, 3]           9,248
      BatchNorm2d-12             [-1, 32, 3, 3]              64
AdaptiveAvgPool2d-13             [-1, 32, 1, 1]               0
           Conv2d-14             [-1, 1

C:\Users\ic016726\AppData\Local\Temp\ipykernel_28120\3380533065.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [10]:
optimizer = optim.SGD(model_2.parameters(),lr=0.01,momentum=0.9)

for epoch in range(1,10):
  train(model_2,device, train_loader, optimizer,epoch)
  test(model_2,device, train_loader)

  0%|          | 0/3750 [00:00<?, ?it/s]C:\Users\ic016726\AppData\Local\Temp\ipykernel_28120\3380533065.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.16939619183540344 batch_id=3749: 100%|██████████| 3750/3750 [00:26<00:00, 143.60it/s]  



 Epoch No: 1
Train set: Average loss: 0.0000, Accuracy: 58327/60000 (97.21%)
Test set: Average loss: 0.0400, Accuracy:59277/60000 (98.80%)



loss=0.004079877864569426 batch_id=3749: 100%|██████████| 3750/3750 [00:26<00:00, 139.81it/s]  



 Epoch No: 2
Train set: Average loss: 0.0000, Accuracy: 59300/60000 (98.83%)
Test set: Average loss: 0.0251, Accuracy:59520/60000 (99.20%)



loss=0.052632950246334076 batch_id=3749: 100%|██████████| 3750/3750 [00:26<00:00, 143.19it/s]  


...


 Epoch No: 7
Train set: Average loss: 0.0000, Accuracy: 59689/60000 (99.48%)
Test set: Average loss: 0.0094, Accuracy:59847/60000 (99.75%)



loss=0.025746427476406097 batch_id=3749: 100%|██████████| 3750/3750 [00:27<00:00, 137.64it/s]  



 Epoch No: 8
Train set: Average loss: 0.0000, Accuracy: 59726/60000 (99.54%)
Test set: Average loss: 0.0089, Accuracy:59858/60000 (99.76%)



loss=0.00029404566157609224 batch_id=3749: 100%|██████████| 3750/3750 [00:28<00:00, 129.58it/s]



 Epoch No: 9
Train set: Average loss: 0.0000, Accuracy: 59776/60000 (99.63%)
Test set: Average loss: 0.0106, Accuracy:59820/60000 (99.70%)



In [11]:
class Net_v3(nn.Module):
  def __init__(self):
    super(Net_v3,self).__init__()
    self.conv1 = nn.Conv2d(1,8,3,stride=1,padding=1) # input_size = 28, output_size = 28, RF = 3,
    self.conv1_bn = nn.BatchNorm2d(8)
    self.conv2 = nn.Conv2d(8,16,3,stride=1,padding=0)# input_size = 28, output_size = 26, RF = 5
    self.conv2_bn = nn.BatchNorm2d(16)
    self.drop1 = nn.Dropout2d(0.05)
    self.pool1 = nn.MaxPool2d(2,2)                    # input_size = 26, output_size = 13, RF = 6
    self.conv3 = nn.Conv2d(16,16,3,stride=1,padding=1)# input_size = 13, output_size = 13, RF = 10
    self.conv3_bn = nn.BatchNorm2d(16)
    self.drop2 = nn.Dropout2d(0.05)
    self.conv4 = nn.Conv2d(16,32,3,stride=1,padding=0)# input_size = 13, output_size = 11, RF = 14
    self.conv4_bn = nn.BatchNorm2d(32)
    self.drop3 = nn.Dropout2d(0.05)
    self.pool2 = nn.MaxPool2d(2,2)                      # input_size = 11, output_size = 5, RF = 16
    self.conv5 = nn.Conv2d(32,32,3,stride=1,padding=0)# input_size = 5, output_size = 3, RF = 24
    self.conv5_bn = nn.BatchNorm2d(32)
    self.pool3 = nn.AdaptiveAvgPool2d((1, 1))  # reduces the spatial dimensions to 1x1
    self.conv6 = nn.Conv2d(32,10,1)                   # input_size = 3, output_size = 1, RF = 40

  def forward(self,x):
    x = self.pool1(F.relu(self.drop1(self.conv2_bn(self.conv2(F.relu(self.conv1_bn(self.conv1(x))))))))
    x = self.pool2(F.relu(self.drop3(self.conv4_bn(self.conv4(F.relu(self.drop2(self.conv3_bn(self.conv3(x)))))))))
    x = self.pool3(F.relu(self.conv5_bn(self.conv5(x))))
    x = self.conv6(x)
    x = x.view(-1,10)
    return F.log_softmax(x)


In [12]:
model_3 = SimpleCNN18K().to(device)
summary(model_3,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 26, 26]           1,168
       BatchNorm2d-4           [-1, 16, 26, 26]              32
         Dropout2d-5           [-1, 16, 26, 26]               0
         MaxPool2d-6           [-1, 16, 13, 13]               0
            Conv2d-7           [-1, 16, 13, 13]           2,320
       BatchNorm2d-8           [-1, 16, 13, 13]              32
         Dropout2d-9           [-1, 16, 13, 13]               0
           Conv2d-10           [-1, 32, 11, 11]           4,640
      BatchNorm2d-11           [-1, 32, 11, 11]              64
        Dropout2d-12           [-1, 32, 11, 11]               0
        MaxPool2d-13             [-1, 32, 5, 5]               0
           Conv2d-14             [-1, 3

d:\Projects\ERAV3\TunningCNN\SimpleCNN18K.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [13]:
optimizer = optim.SGD(model_3.parameters(),lr=0.01,momentum=0.9)

for epoch in range(1,11):
  train(model_3,device, train_loader, optimizer,epoch)
  test(model_3,device, train_loader)

loss=0.10214698314666748 batch_id=3749: 100%|██████████| 3750/3750 [00:32<00:00, 116.00it/s]  



 Epoch No: 1
Train set: Average loss: 0.0000, Accuracy: 57203/60000 (95.34%)
Test set: Average loss: 0.0393, Accuracy:59279/60000 (98.80%)



loss=0.009115763939917088 batch_id=3749: 100%|██████████| 3750/3750 [00:29<00:00, 126.99it/s]  



 Epoch No: 2
Train set: Average loss: 0.0000, Accuracy: 58948/60000 (98.25%)
Test set: Average loss: 0.0312, Accuracy:59414/60000 (99.02%)



loss=0.00018750519666355103 batch_id=3749: 100%|██████████| 3750/3750 [00:30<00:00, 122.73it/s]


...


 Epoch No: 8
Train set: Average loss: 0.0000, Accuracy: 59451/60000 (99.08%)
Test set: Average loss: 0.0125, Accuracy:59777/60000 (99.63%)



loss=0.020221835002303123 batch_id=3749: 100%|██████████| 3750/3750 [00:28<00:00, 133.70it/s]  



 Epoch No: 9
Train set: Average loss: 0.0000, Accuracy: 59462/60000 (99.10%)
Test set: Average loss: 0.0144, Accuracy:59735/60000 (99.56%)



loss=0.01322183571755886 batch_id=3749: 100%|██████████| 3750/3750 [00:29<00:00, 128.06it/s]   



 Epoch No: 10
Train set: Average loss: 0.0000, Accuracy: 59491/60000 (99.15%)
Test set: Average loss: 0.0107, Accuracy:59807/60000 (99.68%)

